In [23]:
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import pandas as pd

# Import the AnimalShelter class from your CRUD Python module
from PythonModule import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password1234"

# Connect to the database via the CRUD Module
db = AnimalShelter(username, password)

# Read data from the database and convert it to a DataFrame
df = pd.DataFrame.from_records(db.read({}))

# MongoDB returns the '_id' column, which is not valid for DataTable, so remove it
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo and encode it to base64
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='logo-id',
                     style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
            href='https://www.snhu.edu'
        ),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.P("Created by: Branden Boehnke", id='creator-id', style={'text-align': 'center'}),
    ]),
  
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        ),
    ]),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,
                         style_table={'overflowX': 'auto'},
                         style_cell={'textAlign': 'left'},
                         style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                         style_data_conditional=[]
                         ),
    html.Br(),
    html.Hr(),
   
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # If no filter or reset is selected, show all data
    if filter_type is None or filter_type == 'RESET':
        return df.to_dict('records')
    else:
        # Define filter criteria based on the selected rescue type
        rescue_filters = {
            'Water Rescue': {
                'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
                'sex': 'Intact Female',
                'age_min': 26,
                'age_max': 156
            },
            'Mountain or Wilderness Rescue': {
                'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
                'sex': 'Intact Male',
                'age_min': 26,
                'age_max': 156
            },
            'Disaster or Individual Tracking': {
                'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
                'sex': 'Intact Male',
                'age_min': 20,
                'age_max': 300
            }
        }

        filter_criteria = rescue_filters.get(filter_type, {})

        # Filter data based on the selected criteria
        df_filtered = df[
            (df['breed'].isin(filter_criteria.get('breeds', []))) &
            (df['sex_upon_outcome'] == filter_criteria.get('sex', '')) &
            (df['age_upon_outcome_in_weeks'] >= filter_criteria.get('age_min', 0)) &
            (df['age_upon_outcome_in_weeks'] <= filter_criteria.get('age_max', float('inf')))
        ]
        
        return df_filtered.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' not in dff.columns:
        return []
    
    # Create a pie chart showing the top breeds
    top_n = 10  # Set your desired number of top breeds
    top_breeds = dff['breed'].value_counts().nlargest(top_n).index

    dff_filtered = dff[dff['breed'].isin(top_breeds)]
    
    return [
        dcc.Graph(            
            figure=px.pie(
                dff_filtered,
                names='breed',
                title=f'Top {top_n} Preferred Breeds',
                hole=0.0,
                color_discrete_sequence=px.colors.sequential.RdBu,
                labels={'breed': 'Breed'},
            )
        )    
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Highlight selected columns in the DataTable
    if selected_columns is None:
        return []
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):  
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)

    selected_row_index = 0
    if selected_rows:
        selected_row_index = selected_rows[0]

    # Get latitude and longitude for the selected row
    lat, long = dff.loc[selected_row_index, 'location_lat'], dff.loc[selected_row_index, 'location_long']

    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, long], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, long], children=[
            dl.Tooltip(dff['breed']),        
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.loc[selected_row_index, 'name'])
            ])
        ])
    ])]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:13362/
